In [24]:
import os


In [25]:
pwd


'/Users/gayatrimanjarisahu/mlproject/kidney_disease'

In [3]:
os.chdir("../")

In [61]:
pwd


'/Users/gayatrimanjarisahu/mlproject/kidney_disease'

In [62]:
#entity is return type of data ingestion function
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [63]:
from pathlib import Path

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

In [64]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [65]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [68]:
import os
import zipfile
import shutil
import gdown
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

In [69]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        # Define the __MACOSX path
        macosx_path = os.path.join(unzip_path, '__MACOSX')
    
       # Remove the __MACOSX directory if it exists
        if os.path.exists(macosx_path):
            shutil.rmtree(macosx_path)
            print(f"Removed {macosx_path}")
        else:
             print(f"{macosx_path} not found, nothing to remove")    

In [70]:
from box import Box
from box.exceptions import BoxValueError
from ensure import ensure_annotations
from box import ConfigBox


In [71]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    

except Exception as e:
    raise e

[2024-07-11 19:56:58,867: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-11 19:56:58,870: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-11 19:56:58,872: INFO: common: created directory at: artifacts]
[2024-07-11 19:56:58,872: INFO: common: created directory at: artifacts/data_ingestion]
[2024-07-11 19:56:58,873: INFO: 844985200: Downloading data from https://drive.google.com/file/d/1Q3pCzSEMDos8c9iR3CapPSXQR6i-sL13/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1Q3pCzSEMDos8c9iR3CapPSXQR6i-sL13
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1Q3pCzSEMDos8c9iR3CapPSXQR6i-sL13&confirm=t&uuid=e01538a2-b305-4e4d-bb38-db1bdbc9b5a5
To: /Users/gayatrimanjarisahu/mlproject/kidney_disease/artifacts/data_ingestion/data.zip
100%|██████████| 83.1M/83.1M [00:02<00:00, 29.4MB/s]

[2024-07-11 19:57:06,594: INFO: 844985200: Downloaded data from https://drive.google.com/file/d/1Q3pCzSEMDos8c9iR3CapPSXQR6i-sL13/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Removed artifacts/data_ingestion/kidney-ct-scan-image/__MACOSX


In [ ]:
#downlod and unzip the file